In [1]:
import logging
import numpy as np  
from gensim.models import Doc2Vec
import os
import codecs

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',level=logging.INFO)

WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.


In [2]:
dbow = Doc2Vec.load("/home/laampt/nlp/xnews/docsent_v1.0/docsent_dbow_5ns_100features_1mw_r20")

In [3]:
print dbow

Doc2Vec(dbow+w,d100,n5,hs,w8,mc2,s0.001,t4)


## Loading trained models

In [4]:
from sklearn.externals import joblib

In [5]:
svc = joblib.load("/home/laampt/nlp/xnews/docsent_v1.0/svc_c10.pkl")
lr = joblib.load("/home/laampt/nlp/xnews/docsent_v1.0/lr_c100.pkl")
gbc = joblib.load("/home/laampt/nlp/xnews/docsent_v1.0/gbc_e200.pkl")

## Helpers

In [23]:
def get_a1_score(doc, dbow=dbow, relevant_score=0.5, isVec=False, cv=3):
    scores = []
    for i in range(cv):
        vec = dbow.infer_vector(doc.lower().strip().split()) if not isVec else doc
        sims = filter(lambda x: x[1] > relevant_score, dbow.docvecs.most_similar([vec]))
        vc_score = 1.0 * np.sum([x[1] for x in sims if x[0].startswith(u"vc_")]) / (np.sum([x[1] for x in sims]) + 0.001)
        scores += [vc_score]
    return np.mean(scores)

In [32]:
get_a1_score(u"tôi yêu đảng cộng_sản việt_nam muôn năm", dbow)

0.067201553003725628

In [33]:
get_a1_score(dbow.docvecs['vc_100'], dbow, isVec=True)

0.99986322145953854

## Predict API

In [34]:
def get_docsent_score(vec, svc=svc,lr=lr, gbc=gbc):
    l1_score = get_a1_score(vec, isVec=True)
    y_predict = 0
    if l1_score < 0.1:
        y_predict = 0
    elif l1_score > 0.9:
        y_predict = 1
    else:
        y_predict = 1 if np.mean([svc.predict(vec)[0], lr.predict(vec)[0], gbc.predict(vec)[0]]) > 0.5 else 0
    return y_predict

In [ ]:
%%time

pos_num = 5090
y_pred = []

for i in range(pos_num):
    v_b = dbow.docvecs["vc_{}".format(i)]
    if v_b.shape[0] == 100: 
        y_pred += [get_docsent_score(v_b)]
    else:
        pass

In [39]:
# test on 5090 black docs
print 1. * np.sum(y_pred) / pos_num

0.997249508841


In [40]:
# can be used with tfidf in version 0.5